In [190]:
import sklearn
import pandas as pd
import numpy as np

In [191]:
df=pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv')
df.columns=list('ABCDEFGHI')
df

,A,B,C,D,E,F,G,H,I
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0
3,0,137,40,35,168,43.1,2.288,33,1
4,5,116,74,0,0,25.6,0.201,30,0
...,...,...,...,...,...,...,...,...,...
762,10,101,76,48,180,32.9,0.171,63,0
763,2,122,70,27,0,36.8,0.340,27,0
764,5,121,72,23,112,26.2,0.245,30,0
765,1,126,60,0,0,30.1,0.349,47,1


In [192]:
df=df.drop([12,7, 227, 246, 285, 408, 583, 654, 752])

In [193]:
from sklearn.model_selection import train_test_split
x,x1,y,y1=train_test_split(df.drop('I',axis=1),df['I'],test_size=0.2)
x=x.values
y=y.values
x1=x1.values

In [194]:
from sklearn.tree import DecisionTreeClassifier
d=DecisionTreeClassifier()
d.fit(x,y)

DecisionTreeClassifier()

In [195]:
res2=d.predict(x1)

In [196]:
(res2==y1).sum()/len(y1)

0.743421052631579

In [197]:
class Node:
    def __init__(self,p):
        self.feature=None
        self.threshold=None
        self.pred=p
        self.prediction=np.argmax(p)
        self.left=None
        self.right=None


class DecisionTree:
    def fit(self,x,y):
        self.n_classes=len(set(y))
        self.tree=self.growtree(x,y,0)
        self.n_features=x.shape[1]
        
    def split1(self, X, y):
        m = y.size
        if m <= 1:
            return None, None
        num_parent = [np.sum(y == c) for c in range(self.n_classes)]
        best_gini = 1.0 - sum((n / m) ** 2 for n in num_parent)
        best_idx, best_thr = None, None
        for idx in range(x.shape[1]):
            thresholds, classes = zip(*sorted(zip(X[:, idx], y)))
            num_left = [0] * self.n_classes
            num_right = num_parent.copy()
            for i in range(1, m):
                c = classes[i - 1]
                num_left[c] += 1
                num_right[c] -= 1
                gini_left = 1.0 - sum(
                    (num_left[x] / i) ** 2 for x in range(self.n_classes)
                )
                gini_right = 1.0 - sum(
                    (num_right[x] / (m - i)) ** 2 for x in range(self.n_classes)
                )
                gini = (i * gini_left + (m - i) * gini_right) / m
                if thresholds[i] == thresholds[i - 1]:
                    continue
                if gini < best_gini:
                    best_gini = gini
                    best_idx = idx
                    best_thr = (thresholds[i] + thresholds[i - 1]) / 2
        return best_idx, best_thr
                    
    def growtree(self,x,y,depth):
        n=[np.sum(y==i) for i in range(self.n_classes)]
        node=Node(n)
        ind,t=self.split1(x,y)
        if(depth<200):
            if(ind is not None):
                node.feature=ind
                node.threshold=t
                a=(x[:,ind]<t)
                node.left=self.growtree(x[a],y[a],depth+1)
                node.right=self.growtree(x[~a],y[~a],depth+1)
        return node
    
    def predict(self,x):
        return [self.predict1(i) for i in x]
    
    def predict1(self,i):
        cur=self.tree
        while(cur.right):
            if(i[cur.feature]>cur.threshold):
                cur=cur.right
            else:
                cur=cur.left
        return cur.prediction       

In [198]:
d=DecisionTree()
d.fit(x,y)

In [199]:
result=d.predict(x1)

In [200]:
(result==y1).sum()/len(y1)

0.7302631578947368

In [201]:
len(y1)

152